<img src=https://static.wixstatic.com/media/f6bd9a_6a972d9b31324653bf198c38ce94339e~mv2.png>

# Carrusel de RDBMS
## Ejercicios de SQL resueltos

En este cuaderno resuelvo los ejercicios del 31 al 40 del documento: 'Desarrollo de Aplicaciones en Entornos de Cuarta Generación y Herramientas CASE: Ejercicios de SQL.'

Para ello utilizo una version de la base de datos modificada para ser compatible en esta ocasion, para DB2, siguiendo la estructura del libro.

En esta ocasion el  diagrama de Entidad-Relacion no esta disponible, pues no pude descargar la version de escritorio de DB2, por lo que uso **DB2 on cloud** . El Script para cargar la base de datos (que permite, se pueda replicar el ejercicio) se encuentran disponibles en el mismo repositorio.

#### Celda para configurar el entorno

In [ ]:
#!pip install ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
#!pip install sqlalchemy==2.0
#!pip install pandas

#### Importamos librerias y se realiza la conexion utilizando un archivo json con parte de la configuracion de la base de datos

In [1]:
import ibm_db
import ibm_db_dbi
import json
import pandas as pd

In [2]:
# Cargar los detalles de la conexión desde un archivo de configuración
with open('config21.json', 'r') as f:
    config = json.load(f)

In [3]:
# Extraer los datos de conexión del objeto JSON
dsn_hostname = config['hostname']
dsn_uid = config['uid']
dsn_pwd = config['pwd']
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = config['database']
dsn_port = config['port']
dsn_protocol = config['protocol']
dsn_security = config['security']

In [4]:
# formar la cadena
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

#### Realizamos una prueba y se Inspecciona la base de datos 

In [5]:
# realizar la conexion
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connexion realizada: ")

except:
    print ("No se pudo realizar la conexion: ", ibm_db.conn_errormsg() )

Connexion realizada: 


In [6]:
# para usar con pandas
pconn = ibm_db_dbi.Connection(conn)

In [7]:
QUERY = """
SELECT TABNAME
FROM SYSCAT.TABLES
WHERE TABSCHEMA = CURRENT SCHEMA;
"""
df = pd.read_sql_query(QUERY, pconn)
df

,TABNAME
0,HOSPITAL
1,SALA
2,PLANTILLA
3,OCUPACION
4,DOCTOR
5,ENFERMO
6,EMPLEADO
7,DEPARTAMENTO


#### Inspeccion a las tablas de la base de datos 
#### Con la finalidad de observar su estructura y tipo de datos

In [8]:
for valor in df['TABNAME']:
    QUERY = "SELECT COLNAME, TYPENAME, LENGTH, SCALE FROM SYSCAT.COLUMNS WHERE TABNAME = "+ "'"+ f'{valor}'+"'"+' AND TABSCHEMA = CURRENT SCHEMA;'
    df1 = pd.read_sql_query(QUERY, pconn)
    print(f'El contenido de la tabla {valor}\n')
    print(df1)
    print()

El contenido de la tabla HOSPITAL

        COLNAME TYPENAME  LENGTH  SCALE
0     DIRECCION  VARCHAR      20      0
1  HOSPITAL_COD  INTEGER       4      0
2        NOMBRE  VARCHAR      10      0
3      NUM_CAMA  INTEGER       4      0
4      TELEFONO  VARCHAR       8      0

El contenido de la tabla SALA

        COLNAME TYPENAME  LENGTH  SCALE
0  HOSPITAL_COD  INTEGER       4      0
1        NOMBRE  VARCHAR      20      0
2      NUM_CAMA  INTEGER       4      0
3      SALA_COD  INTEGER       4      0
4       SALA_ID  INTEGER       4      0

El contenido de la tabla PLANTILLA

        COLNAME TYPENAME  LENGTH  SCALE
0      APELLIDO  VARCHAR      15      0
1   EMPLEADO_NO  INTEGER       4      0
2       FUNCION  VARCHAR      10      0
3  HOSPITAL_COD  INTEGER       4      0
4       SALARIO  INTEGER       4      0
5      SALA_COD  INTEGER       4      0
6         TURNO  VARCHAR       1      0

El contenido de la tabla OCUPACION

        COLNAME TYPENAME  LENGTH  SCALE
0          CAMA  IN

### Resolucion de los ejercicios numerados del 31 al 40

##### COMBINACIONES DE TABLAS

##### 31. Listar, a partir de las tablas EMPLEADO y DEPARTAMENTO, el nombre de cada empleado, su oficio, su numero de departamento y el nombre del departamento donde trabajan

In [9]:
QUERY = """
SELECT E.APELLIDO, E.OFICIO, E.DEPT_NO, D.DNOMBRE
FROM EMPLEADO AS E
INNER JOIN DEPARTAMENTO AS D
ON E.DEPT_NO = D.DEPT_NO
;
"""
df = pd.read_sql_query(QUERY, pconn)
df

,APELLIDO,OFICIO,DEPT_NO,DNOMBRE
0,Sanchez,Empleado,20,INVESTIGACIÓN
1,Arroyo,Vendedor,30,VENTAS
2,Sala,Vendedor,30,VENTAS
3,Jimenez,Director,20,INVESTIGACIÓN
4,Arenas,Vendedor,30,VENTAS
5,Negro,Director,30,VENTAS
6,Cerezo,Director,10,CONTABILIDAD
7,Gil,Analista,20,INVESTIGACIÓN
8,Rey,Presidente,10,CONTABILIDAD
9,Tovar,Vendedor,30,VENTAS


##### 32. Seleccionar los nombres, profesiones y localidades de los departamentos donde trabajan los Analistas.

In [10]:
QUERY = """
SELECT E.APELLIDO, E.OFICIO, D.LOC 
FROM EMPLEADO AS E
INNER JOIN DEPARTAMENTO AS D
ON E.DEPT_NO = D.DEPT_NO
WHERE E.DEPT_NO = (SELECT DISTINCT(DEPT_NO) FROM EMPLEADO WHERE OFICIO = 'Analista');
"""
df = pd.read_sql_query(QUERY, pconn)
df

,APELLIDO,OFICIO,LOC
0,Sanchez,Empleado,BILBAO
1,Jimenez,Director,BILBAO
2,Gil,Analista,BILBAO
3,Alonso,Empleado,BILBAO
4,Fernandez,Analista,BILBAO


##### 33. Se desea conocer el nombre y oficio de todos aquellos empleados que trabajan en Madrid. La salida deberá estar ordenada por el oficio.

In [11]:
QUERY = """
SELECT E.APELLIDO, E.OFICIO, D.LOC
FROM EMPLEADO AS E
INNER JOIN DEPARTAMENTO AS D
ON E.DEPT_NO = D.DEPT_NO
WHERE UPPER(LOC) = 'MADRID'
ORDER BY E.OFICIO;
"""
df = pd.read_sql_query(QUERY, pconn)
df

,APELLIDO,OFICIO,LOC
0,Cerezo,Director,MADRID
1,Muñoz,Empleado,MADRID
2,Rey,Presidente,MADRID


##### 34. se desea conocer cuantos empleados existen en cada departamento. Devolviendo una salida como la que se presenta (deberá estar ordenada por el numero de empleados descendéntemente).


|NUM-DEP|DEPARTAMENTO|N-EMPL|
|:---:|:-----------:|:-------------:|
|30|VENTAS|6|
|20|INVESTIGACIÓN|5|
|10|CONTABILIDAD|3|





In [12]:
QUERY = """
SELECT D.DEPT_NO AS NUM_DEP, D.DNOMBRE AS DEPARTAMENTO, COUNT(E.EMP_NO) AS N_EMPL
FROM DEPARTAMENTO AS D
INNER JOIN EMPLEADO AS E
ON E.DEPT_NO = D.DEPT_NO
GROUP BY D.DEPT_NO, D.DNOMBRE
ORDER BY N_EMPL DESC;
"""
df = pd.read_sql_query(QUERY, pconn)
df

,NUM_DEP,DEPARTAMENTO,N_EMPL
0,30,VENTAS,6
1,20,INVESTIGACIÓN,5
2,10,CONTABILIDAD,3


##### 35. Se desea conocer, tanto para el departamento de VENTAS, como para el de CONTABILIDAD, su maximo, su mínimo y su media salarial, así como el numero de empleados en cada departamento. La salida deberá estar ordenada por el nombre del departamento, y se deberá presentar como la siguiente:
 
|DNOMBRE|MAXIMO|MÍNIMO|MEDIA|N-EMPL|
|:---------------:|:------------:|:------------:|:------------:|:-------:|
|CONTABILIDAD|650000|169000|379166.667|3|
|VENTAS|370500|123500|203666.667|6|



In [13]:
QUERY = """
SELECT D.DNOMBRE, MAX(E.SALARIO) AS MAXIMO, MIN(E.SALARIO) AS MINIMO, AVG(E.SALARIO) AS MEDIA, COUNT(E.EMP_NO) AS N_EMPL
FROM DEPARTAMENTO AS D
INNER JOIN EMPLEADO AS E
ON D.DEPT_NO = E.DEPT_NO
GROUP BY D.DNOMBRE
HAVING UPPER(D.DNOMBRE) IN ('VENTAS', 'CONTABILIDAD')
ORDER BY D.DNOMBRE;
"""
df = pd.read_sql_query(QUERY, pconn)
df

,DNOMBRE,MAXIMO,MINIMO,MEDIA,N_EMPL
0,CONTABILIDAD,650000,169000,379166,3
1,VENTAS,370500,123500,203666,6


##### 36. Se desea conocer el maximo salario que existe en cada sala de cada hospital, dando el resultado como sigue
|HOSPITAL|SALA|MAXIMO|
|:-------------:|:------------:|:-------------:|
|General|Cardiología|3379000|
|La Paz|Maternidad|2210000|
|La Paz|Psiquiátrico|2005000|
|La Paz|Recuperación|2119000|
|Provincial|Psiquiátrico|2755000|
|San Carlos|Cardiología|2210000|
|San Carlos|Recuperación|2522000|

In [14]:
QUERY = """
SELECT H.NOMBRE AS HOSPITAL, S.NOMBRE AS SALA, MAX(P.SALARIO) AS MAXIMO
FROM HOSPITAL AS H
INNER JOIN SALA AS S ON H.HOSPITAL_COD = S.HOSPITAL_COD
INNER JOIN PLANTILLA AS P ON (S.SALA_COD = P.SALA_COD AND S.HOSPITAL_COD = P.HOSPITAL_COD)
GROUP BY H.NOMBRE, S.NOMBRE;
"""
df = pd.read_sql_query(QUERY, pconn)
df

,HOSPITAL,SALA,MAXIMO
0,General,Cardiología,3379000
1,La Paz,Maternidad,3000000
2,La Paz,Psiquiátrico,2005000
3,La Paz,Recuperación,2119000
4,Provincial,Psiquiátrico,2755000
5,San Carlos,Cardiología,2210000
6,San Carlos,Recuperación,2522000


##### 37. Se desea obtener un resultado como el que aparece, en el que se presenta el numero, nombre y oficio de cada empleado de nuestra empresa que tiene jefe, y lo mismo de su jefe directo. La salida debe estar ordenadA por el nombre del empleado
|EMPLEADO|NOMBRE|OFICIO|JEFE|NOMBRE|OFICIO|
|:-----------------:|:----------------:|:---------------:|:--------:|:-------------:|:--------------:|
|7876|ALONSO|EMPLEADO|7788|GIL|ANALISTA|
|7499|ARROYO|VENDEDOR|7698|NEGRO|DIRECTOR|
|7782|CEREZO|DIRECTOR|7839|REY|PRESIDENTE|
|7902|FERNANDEZ|ANALISTA|7566|JIMENEZ|DIRECTOR|
|7788|GIL|ANALISTA|7566|JIMENEZ|DIRECTOR|
|7566|JIMENEZ|DIRECTOR|7839|REY|PRESIDENTE|
|7900|JIMENO|EMPLEADO|7698|NEGRO|DIRECTOR|
|7654|MARTIN|VENDEDOR|7698|NEGRO|DIRECTOR|
|7934|MUÑOZ|EMPLEADO|7782|CEREZO|DIRECTOR|
|7698|NEGRO|DIRECTOR|7839|REY|PRESIDENTE|
|7521|SALA|VENDEDOR|7698|NEGRO|DIRECTOR|
|7369|SANCHEZ|EMPLEADO|7902|FERNANDEZ|ANALISTA|
|7844|TOVAR|VENDEDOR|7698|NEGRO|DIRECTOR|
 

In [15]:
QUERY = """
SELECT  E.EMP_NO AS EMPLEADO, E.APELLIDO AS NOMBRE, E.OFICIO, E.DIR AS JEFE,
        E2.APELLIDO AS NOMBRE, E2.OFICIO  AS OFICIO
FROM EMPLEADO AS E, EMPLEADO AS E2
WHERE E.DIR = E2.EMP_NO
ORDER BY 2;
"""
df = pd.read_sql_query(QUERY, pconn)
df

,EMPLEADO,NOMBRE,OFICIO,JEFE,NOMBRE,OFICIO
0,7876,Alonso,Empleado,7788,Gil,Analista
1,7654,Arenas,Vendedor,7698,Negro,Director
2,7499,Arroyo,Vendedor,7698,Negro,Director
3,7782,Cerezo,Director,7839,Rey,Presidente
4,7902,Fernandez,Analista,7566,Jimenez,Director
5,7788,Gil,Analista,7566,Jimenez,Director
6,7566,Jimenez,Director,7839,Rey,Presidente
7,7900,Jimeno,Empleado,7698,Negro,Director
8,7934,Muñoz,Empleado,7782,Cerezo,Director
9,7698,Negro,Director,7839,Rey,Presidente


##### 38. Se desea conocer, para todos los departamentos existentes, el mínimo salario de cada departamento, mostrando el resultado como aparece. Para el muestreo del mínimo salario, no queremos tener en cuenta a las personas con oficio de EMPLEADO. La salida estará ordenada por el salario descendéntemente.

|DEPARTAMENTO| MÍNIMO|
|:-----------------------:|:----------:|
|INVESTIGACIÓN| 386750|
|CONTABILIDAD| 318500|
|VENTAS| 162500|
|OPERACIONES| 0|

In [16]:
QUERY = """
SELECT D.DNOMBRE AS DEPARTAMENTO, MIN(IFNULL(E.SALARIO,0)) AS MINIMO
FROM DEPARTAMENTO AS D
FULL OUTER JOIN EMPLEADO AS E
ON D.DEPT_NO = E.DEPT_NO
WHERE UPPER(E.OFICIO) <> 'EMPLEADO' OR E.OFICIO IS NULL
GROUP BY D.DNOMBRE
ORDER BY MINIMO DESC;
"""
df = pd.read_sql_query(QUERY, pconn)
df

,DEPARTAMENTO,MINIMO
0,INVESTIGACIÓN,386750
1,CONTABILIDAD,318500
2,VENTAS,162500
3,OPERACIONES,0


##### 39. Se desea sacar un listado con el mismo formato del ejercicio 37 tal y como el que aparece, pero ahora también se desea sacar al jefe de la empresa como empleado, pues en el listado del citado ejercicio no aparecía.

In [17]:
QUERY = """
SELECT  E.EMP_NO AS EMPLEADO, E.APELLIDO AS NOMBRE, E.OFICIO, E.DIR AS JEFE,
        E2.APELLIDO AS NOMBRE,E2.OFICIO  AS OFICIO
FROM EMPLEADO AS E
LEFT OUTER JOIN EMPLEADO AS E2
ON E.DIR = E2.EMP_NO
ORDER BY 2;
"""
df = pd.read_sql_query(QUERY, pconn)
df

,EMPLEADO,NOMBRE,OFICIO,JEFE,NOMBRE,OFICIO
0,7876,Alonso,Empleado,7788.0,Gil,Analista
1,7654,Arenas,Vendedor,7698.0,Negro,Director
2,7499,Arroyo,Vendedor,7698.0,Negro,Director
3,7782,Cerezo,Director,7839.0,Rey,Presidente
4,7902,Fernandez,Analista,7566.0,Jimenez,Director
5,7788,Gil,Analista,7566.0,Jimenez,Director
6,7566,Jimenez,Director,7839.0,Rey,Presidente
7,7900,Jimeno,Empleado,7698.0,Negro,Director
8,7934,Muñoz,Empleado,7782.0,Cerezo,Director
9,7698,Negro,Director,7839.0,Rey,Presidente


##### 40. Se desea obtener un listado como el que aparece, es decir, como el del ejercicio 37, pero obteniendo además todos aquellos empleados que no son jefes de nadie en la parte de jefe, para que se vea gráficamente que no tienen subordinados a su cargo.

In [18]:
QUERY = """
SELECT  E.EMP_NO AS EMPLEADO, E.APELLIDO AS NOMBRE, E.OFICIO, E.DIR AS JEFE,
        E2.APELLIDO AS NOMBRE,E2.OFICIO  AS OFICIO
FROM EMPLEADO AS E
FULL OUTER JOIN EMPLEADO AS E2
ON E.DIR = E2.EMP_NO
ORDER BY 2;
"""
df = pd.read_sql_query(QUERY, pconn)
df

,EMPLEADO,NOMBRE,OFICIO,JEFE,NOMBRE,OFICIO
0,7876.0,Alonso,Empleado,7788.0,Gil,Analista
1,7654.0,Arenas,Vendedor,7698.0,Negro,Director
2,7499.0,Arroyo,Vendedor,7698.0,Negro,Director
3,7782.0,Cerezo,Director,7839.0,Rey,Presidente
4,7902.0,Fernandez,Analista,7566.0,Jimenez,Director
5,7788.0,Gil,Analista,7566.0,Jimenez,Director
6,7566.0,Jimenez,Director,7839.0,Rey,Presidente
7,7900.0,Jimeno,Empleado,7698.0,Negro,Director
8,7934.0,Muñoz,Empleado,7782.0,Cerezo,Director
9,7698.0,Negro,Director,7839.0,Rey,Presidente


##### Se cierra la conexion

In [19]:
ibm_db.close(conn)

True

# Autor
### Oscar Gutierrez Leal
28 - 03 - 2024